# Timefrequency analysis 
## 1. Calculation of the ERDS values

In [ ]:
import os 
import re 
import json 
import glob
import h5py 
import warnings
import numpy as np
import mne
import matplotlib.pyplot as plt
from matplotlib.colors import TwoSlopeNorm
import pandas as pd

# The relevant directories are identified here and checked for existence.
# These should be changed according to your own file structure.
EPO_DIR = "/Users/lucas.assen/Desktop/Master Thesis/sub_epoch_files"
OUT_DIR = "/Users/lucas.assen/Desktop/Master Thesis/Time_frequency_analysis_3"
BANDS_CSV = "/Users/lucas.assen/Desktop/Master Thesis/Data/my_mne_project/mne_env/IAF_After_channel_rejection.csv"
os.makedirs(OUT_DIR, exist_ok=True)

# We created these constants which can be adjusted if wanted. Decimate is 1 refering to no decimation of the EEG signal. 
# N_JOBS is the number of cpu cores used for paralell processing, this should be tweaked based on what your own system can handle.
# BASE_EPS is a small value to avoid log of zero or negative values, during baseline correction.
DECIM = 1
N_JOBS = 12
BASE_EPS = 1e-15          

# Similar to previous analysis steps wwe use a helper to normilise the participant IDs across the diffferent files.
def norm_pid(v):
    m = re.match(r"^0*?(\d+)(?:\\.0+)?([A-Za-z]*)$", str(v).strip())
    return (m.group(1) + m.group(2)) if m else str(v).strip()

# We created a common frequency grid using the following helper function; the grid is from 0.3 to 35 Hz in steps of 0.5 Hz.
# Even through individual frequency bands are used for the ERDS masks, a common grid is used first to compute the TFRs.
def grid(lo, hi, step=0.5):
    return np.arange(lo, hi + step/2, step)

COMMON_FREQS = grid(0.3, 35.0, 0.5).round(2)

# Next personalised masks based on the individualised frequency bands are created to be used later 
# to select the bins on a individual basis to calculate ERDS.
bands = pd.read_csv(BANDS_CSV)
bands.columns = bands.columns.str.strip().str.lower()
id_col = next(c for c in bands.columns if "id" in c)
bands[id_col] = (bands[id_col].astype(str)
                 .str.replace(r"_.*$", "", regex=True)
                 .str.strip()
                 .apply(norm_pid))

indiv_mask_dict = {}
for _, row in bands.iterrows():
    pid = row[id_col]
    mask = np.zeros_like(COMMON_FREQS, dtype=bool)
    for lo, hi in [(row.delta_low, row.delta_high),
                   (row.theta_low, row.theta_high),
                   (row.alpha_low, row.alpha_high),
                   (row.beta_low, row.beta_high),]:
         mask |= (COMMON_FREQS >= lo) & (COMMON_FREQS <= hi)
    indiv_mask_dict[pid] = mask

# The regions are again defined and stored in a list to be used for regional averaging of the ERD/S.
REGIONS = {"F1": ["AF7","AF3","F7","F5","F3","F1"],
           "F2": ["AF4","AF8","F4","F6","F2","F8"],
           "C" : ["Fz","FC1","FCZ","FC2","C1","Cz","C2","CP1","CPZ","CP2","P1","Pz","P2","POZ"],
           "O" : ["PO7","PO8","PO3","PO4","O1","OZ","O2"],
           "P1": ["TP7","CP5","CP3","P7","P5","P3"],
           "P2": ["CP4","CP6","CP8","P4","P6","P8"],
           "T1": ["FT7","FC5","FC3","T7","C5","C3"],
           "T2": ["FC4","FC6","FC8","C4","C6","T8"],}
REGION_NAMES = list(REGIONS)
N_REG = len(REGION_NAMES)

# The following loop goes through each participants' extracted epochs file stored in the EPO_DIR path folder.
for fn in sorted(os.listdir(EPO_DIR)):
    if not fn.endswith("_segments-epo.fif"):
        continue
    pid = norm_pid(fn.split("_")[0].replace("sub-", ""))
    if pid not in indiv_mask_dict:
        print(f"participant {pid} skipped as they had no individualised mask")
        continue
    
    #Participant's their epoch file is loaded and some metadata columns are renamed for consistency.
    print(f"Starting with participant {pid}")
    epochs = mne.read_epochs(os.path.join(EPO_DIR, fn),
                             preload=True, verbose="ERROR")
    epochs.metadata = epochs.metadata.rename(columns={'block_idx': 'block'})
    meta = epochs.metadata.reset_index(drop=True)

    # freqs is defined as the eaerlier created common frequency grid and n_cycles is set to half of the frequency value at any frequency 
    # to balance temporal and spatial precision.
    freqs  = COMMON_FREQS
    n_cycles = freqs / 2.0

    pr_idx = np.where(meta.kind == "practice")[0]
    m_prep = meta.kind == "prep"
    if pr_idx.size == 0:
        warnings.warn(f"{pid}: no practice epochs");  continue

    # The regions are mapped based on the channels in the region and the channels present in the current participant's data.
    ch_ix = {ch: i for i, ch in enumerate(epochs.ch_names)}
    reg_to_ix = [[ch_ix[c] for c in REGIONS[r] if c in ch_ix] for r in REGION_NAMES]

    # A dummy TFR is run to get the shape of the data cube in order to create the shape of the H5 dataset.
    n_t = epochs[pr_idx[0]:pr_idx[0]+1].compute_tfr("morlet", 
                                                    freqs=freqs,
                                                    n_cycles=n_cycles,
                                                    output="power", 
                                                    decim=DECIM, n_jobs=1).data.shape[-1]

    h5path = os.path.join(OUT_DIR, f"sub-{pid}_tfr_9d.h5")
    with h5py.File(h5path, "w") as h5:
        ds = h5.create_dataset("erds", (len(pr_idx), 8, len(freqs), n_t),
                               dtype="float32", compression="gzip",
                               chunks=(1, 8, len(freqs), n_t))
        h5.create_dataset("freqs", data=freqs)
        h5.create_dataset("times", data=np.arange(n_t)*DECIM/epochs.info["sfreq"])
        h5.create_dataset("indiv_mask", data=indiv_mask_dict[pid])
        h5.create_dataset("region_names",
                          data=np.array(REGION_NAMES, dtype=h5py.string_dtype()))
        h5.attrs.update(event_id=json.dumps(epochs.event_id),
                        sfreq=epochs.info["sfreq"],
                        metric="dB_ERDS")
        
        # The metadata from the epoch fif files is is also stored in a subgroup in the H5 file.
        g_meta = h5.create_group("metadata")
        meta_pr = meta.iloc[pr_idx].reset_index(drop=True)
        str_dt = h5py.string_dtype(encoding="utf-8")
        for col in meta_pr.columns:
            arr = meta_pr[col].to_numpy()
            if arr.dtype.kind in ("O", "U", "S"):          
                data = np.array(arr.astype(str), dtype=object)
                g_meta.create_dataset(col, data=data, dtype=str_dt)
            else:                                          
                g_meta.create_dataset(col, data=arr, dtype=arr.dtype)

        # After the creation of the data structure of the H5 file the loop over the practice epochs is started to calculate the ERDS.
        # Each practice epoch is seperately processed and written to the H5 file to avoid heavy memory load.
        # First the number of prep practice epochs is identified.
        for i, ix in enumerate(pr_idx):
            blk  = int(meta.block.iloc[ix])
            ix_p = np.where(m_prep & (meta.block == blk))[0][0]

            # The baseline is calculated by first running the tfr on the prep epoch and then averaging across the duration of the prep phase.
            base = epochs[ix_p:ix_p+1].compute_tfr("morlet", freqs=freqs, n_cycles=n_cycles,
                                                   output="power", 
                                                   decim=DECIM, 
                                                   n_jobs=N_JOBS).data[0]                                 
            base_reg = np.stack([base[ix_list].mean(axis=0).mean(axis=-1) if ix_list else np.nan
                                 for ix_list in reg_to_ix], axis=0)[:, :, None]                 
            base_reg = np.where(base_reg < BASE_EPS, np.nan, base_reg)

            # Next the power during the practice epoch is calculated in a similar manner, except no averaging over time is done here.
            pow_p = epochs[ix:ix+1].compute_tfr("morlet", freqs=freqs, 
            n_cycles=n_cycles, 
            output="power", 
            decim=DECIM, 
            n_jobs=N_JOBS).data[0]                                   
            pow_reg = np.stack([pow_p[ix_list].mean(axis=0) if ix_list else np.nan
                for ix_list in reg_to_ix], axis=0)
            # The practice power is then baseline corrected using the earlier calculated baseline from the prep phase of the same block.
            ratio  = pow_reg / base_reg  
            #Both options are included here for calculating ERDS in either dB or as a percentage. We chose dB in our analysis but you can switch to percentage if wanted.                       
            #ds[i] = 100 * (ratio - 1)            
            ds[i] = 10.0 * np.log10(ratio)     
            del base, base_reg, pow_p, pow_reg
            gc.collect()

    print("saved the data to the H5 file", h5path)
    del epochs; gc.collect()

print("Done with all participants")

## 2. Extraction of the theta and alpha band ERD/S

In [ ]:
# Following the calculation of the TFR we take the H5 files and transform the data to the final ERD/S output used for further analyses.
# We first define the relevant directeries, please change the paths to your own, also make sure that the code uses the same naming conventions as your files.
TFR_DIR   = "/Users/lucas.assen/Desktop/Master Thesis/Time_frequency_analysis_2"
BANDS_CSV = "/Users/lucas.assen/Desktop/Master Thesis/Data/my_mne_project/mne_env/Individualised_freq_bands_after_chan_reject.csv"
OUT_DIR   = "/Users/lucas.assen/Desktop/Master Thesis/ERDS_band_csv"
os.makedirs(OUT_DIR, exist_ok=True)

# We again use the helper to normalise participant names, in this case, between the TFR_DIR and the BANDS_CSV
def norm_pid(v: str):
    m = re.match(r"^0*?(\d+)(?:\\.0+)?([A-Za-z]*)$", v.strip())
    return (m.group(1) + m.group(2)) if m else v.strip()

# The decode helper is needed to turn the meta data, which was attached in the H5 file as a subgroup, back into readable strings instead of binary data.
def decode(col):
    s = pd.Series(col)
    return s.apply(lambda x: x.decode() if isinstance(x, (bytes, bytearray)) else str(x)).str.strip()

# We recreated the dictionary for individualised band edges from the BANDS_CSV file for each participant.
bands_df = pd.read_csv(BANDS_CSV)
bands_df.columns = bands_df.columns.str.lower().str.strip()
id_col = next(c for c in bands_df.columns if "id" in c)
bands_df[id_col] = bands_df[id_col].astype(str).str.replace(r"_.*$", "", regex=True).apply(norm_pid)

band_edges = {}   
for _, row in bands_df.iterrows():
    band_edges[row[id_col]] = {"delta": (row.delta_low,  row.delta_high),
    "theta": (row.theta_low,  row.theta_high),
    "alpha": (row.alpha_low,  row.alpha_high),
    "beta" : (row.beta_low,   row.beta_high),}

# Afterwards, we looped through all H5 files in the TFR_DIR. Again make sure to change the "sub-*_tfr_9d.h5" in case you use differing naming conventions.
for fp in sorted(glob.glob(os.path.join(TFR_DIR, "sub-*_tfr_9d.h5"))):
    pid = norm_pid(os.path.basename(fp).split("_")[0].replace("sub-", ""))
    
    #A skip is put in place in case the individualised band edges of a participant do not exist, if you have the correct data this should not trigger.
    if pid not in band_edges:
        warnings.warn(f"No band limits for {pid}, therefore this participant is skipped.")
        continue
    
    # The file is loaded and the tree is walked, extracting the erd/s for all frequencies, time points and regions per epoch
    print(f"Currently working on {pid}")
    with h5py.File(fp, "r") as h5:
        erds = h5["erds"][:]                        # (epochs, 8, F, T)
        freqs = h5["freqs"][:]                       # (F,)
        times = h5["times"][:]                       # (T,)
        region_names = [r.decode() if isinstance(r, (bytes, bytearray)) else str(r)
        for r in h5["region_names"][:]]
        meta = {k: h5["metadata"][k][:] for k in h5["metadata"]}
        meta = pd.DataFrame(meta)

    # The meta data is decoded to get the group practice, type and block numbers out for further 
    meta["group"] = decode(meta["group"]).str.rstrip("s").str.title()
    meta["practice_type"] = decode(meta["cond"]).str.upper()
    meta["block"] = meta["block"].astype(int)

    # The individualised frequency mask is created based on the individualised frequency bands in order 
    # to select the right frequency bins for each band for the current participant.
    masks = {band: (freqs >= lo) & (freqs <= hi)
             for band, (lo, hi) in band_edges[pid].items()}

    # Bins for the time points are created in order to create 14 equally sized 5 second periods.
    t_bins = np.arange(0, 70 + 5, 5)
    t_idx  = np.digitize(times, t_bins) - 1

    # A list is prepared which is then saved to a csv data frame. Then for each time point bin in each individualised freq band for each region within each block, 
    # the average ERD/S is calculated and store in a row in the list with the adhering metadata information, i.e., part id, group, block, pract_type, Strecth_k, etc.
    rows = []
    for ep, row_meta in meta.iterrows():
        for r, reg_name in enumerate(region_names):
            reg_mat = erds[ep, r]                    
            for band, msk in masks.items():
                band_pow = np.nanmean(reg_mat[msk, :], axis=0)  
                for tp in range(14):                              
                    seg_val = np.nanmean(band_pow[t_idx == tp])
                    rows.append({"participant_id": pid,
                    "group": row_meta["group"],
                    "block": row_meta["block"],
                    "practice_type": row_meta["practice_type"],
                    "stretch_k": row_meta["stretch_k"],
                    "rating": row_meta["rating"],
                    "region": reg_name,
                    "freq_band": band,
                    "timepoint": tp + 1,    
                    "erds_db": seg_val,})
    df_out = pd.DataFrame(rows)
    out_path = os.path.join(OUT_DIR, f"sub-{pid}_band_region_timeseries.csv")
    df_out.to_csv(out_path, index=False)
    print(f"Done with {pid}")
print("Done with all participants")

## 3. The creation of ERD/S maps

In [ ]:
# Next to the extraction of the ERD/S values for the individualised band, we also use the H5 files to create general visualisations of the TFR for each region and condition
# Make sure your file paths are correct
TFR_DIR = "/Users/lucas.assen/Desktop/Master Thesis/Time_frequency_analysis_2"
OUT_DIR = "/Users/lucas.assen/Desktop/Master Thesis/Grand_ERDS_maps_2"
os.makedirs(OUT_DIR, exist_ok=True)

#These cells are used to give titles to the plots
CELLS = {("Buddhist", "Fa"): "Buddhists_FA",
("Buddhist", "Lk"): "Buddhists_LKM",
("Control",  "Lk"): "Novices_LKM",}

# The freq_max constant is set to 15Hz as the primary interest are the theta and alpha band, thus only plotting the relevant part of the freq frange. 
FREQ_MAX = 15.0   

# The same decoder is needed here to help extract the metadata from the subgroup in the H5 file. 
def decode_series(s: pd.Series) -> pd.Series:
    return (s.apply(lambda x: x.decode() if isinstance(x, (bytes, bytearray)) else str(x))
              .str.strip().str.rstrip("s").str.title())

# The relevant files are located and for each file the relevant information is retrieved, 
# i.e, ERD/s per freq, time point and epoch and the group and condition from the metadata.  
files = sorted(glob.glob(os.path.join(TFR_DIR, "sub-*_tfr_9d.h5")))
if not files:
    raise RuntimeError("No *_tfr_9d.h5 files found")

cell_data, cell_times = {lab: [] for lab in CELLS.values()}, {}
for f in files:
    with h5py.File(f, "r") as h5:
        erds = h5["erds"][:]                       
        freqs = h5["freqs"][:]
        times = h5["times"][:]
        meta = pd.DataFrame({k: h5["metadata"][k][:] for k in h5["metadata"]})
        grp = decode_series(meta["group"])
        cond = decode_series(meta["cond"])

        #All epochs belioging to the same group/condition are stacked and collapsed together.
        for (g, c), label in CELLS.items():
            m = (grp == g) & (cond == c)
            if m.any():
                cell_data[label].append(erds[m].mean(axis=0))   
                cell_times.setdefault(label, times)
                if not np.allclose(times, cell_times[label]):
                    warnings.warn(f"time mismatch in {f}")

# When averaging over epoch a grand average is created by averaging over all participants belonging in the same condition for each time point and frequency
grand = {lab: np.stack(lst).mean(axis=0) for lab, lst in cell_data.items() if lst}
if not grand:
    raise RuntimeError("No grandaverages calculated")

# Afterwards, regional averages across participants for each group and condition are also calculated.
with h5py.File(files[0], "r") as h5:
    region_names = [r.decode() if isinstance(r, bytes) else r for r in h5["region_names"][:]]

freq_mask = freqs <= FREQ_MAX
freqs_sub = freqs[freq_mask]

# Using the following helper function the grand averages and regional averages for each condition and group are plotted and saved as PNGs to the output directory.
def plot_map(Z_full, freqs, times, title, path):
    freq_mask = freqs <= FREQ_MAX
    freqs_sub = freqs[freq_mask]
    Z = Z_full[freq_mask]  
    lo, hi = np.nanpercentile(Z, [2, 98])
    lim = max(abs(lo), abs(hi))
    norm = TwoSlopeNorm(vmin=-lim, vcenter=0, vmax=lim)
    plt.figure(figsize=(6, 4))
    pcm = plt.pcolormesh(times, freqs_sub, Z, shading="auto", cmap="RdBu_r", norm=norm)
    plt.axvline(0, color="k", lw=0.7)
    plt.xlabel("Time [s]"); 
    plt.ylabel("Frequency [Hz]")
    plt.title(title)
    plt.colorbar(pcm, label="ERDS [dB]")
    plt.tight_layout()
    plt.savefig(path, dpi=150)
    plt.close()

# Next, the plot_map helper function is looped to create the grand average and regional plots for each condition and group.
for lab, tensor in grand.items():           
    times = cell_times[lab]
    for reg_idx, reg_name in enumerate(region_names):
        fn = os.path.join(OUT_DIR, f"{lab}_{reg_name}.png")
        plot_map(tensor[reg_idx], freqs, times, f"{lab} · {reg_name}", fn)
    all_regions = tensor.mean(axis=0)          
    fn_all = os.path.join(OUT_DIR, f"{lab}_AllRegions.png")
    plot_map(all_regions, freqs, times, f"{lab} · All Regions", fn_all)

print("The ERD/S plots are saved as PNGs to:", OUT_DIR)

## 4. The extraction of the alpha/theta ratio (ATR)

In [ ]:
# The relevant directories and constants are similar as before.
# Now we do not need a baseline as we want the raw power ratios of alpha and theta, we also can directly save the results to CSV files 
# as we do not need the full TFR data, only the ratios per region, timepoint and block.
EPO_DIR = "/Users/lucas.assen/Desktop/Master Thesis/sub_epoch_files"
OUT_DIR = "/Users/lucas.assen/Desktop/Master Thesis/Time_frequency_analysis"
BANDS_CSV = "/Users/lucas.assen/Desktop/Master Thesis/Data/my_mne_project/mne_env/IAF_After_channel_rejection.csv"
os.makedirs(OUT_DIR, exist_ok=True)

DECIM = 1
N_JOBS = 12

# Again a helper to normilise participant IDs is created and the same normalised frequency grid is used for the tfr computations.
def norm_pid(v):
    m = re.match(r"^0*?(\d+)(?:\\.0+)?([A-Za-z]*)$", str(v).strip())
    return (m.group(1) + m.group(2)) if m else str(v).strip()

def grid(lo, hi, step=0.5):
    return np.arange(lo, hi + step/2, step)
COMMON_FREQS = grid(0.3, 35.0, 0.5).round(2)

# Similar indiv masks are created based on the individualised frequency bands to be used after the tfr computations.
bands = pd.read_csv(BANDS_CSV)
bands.columns = bands.columns.str.strip().str.lower()
id_col = next(c for c in bands.columns if "id" in c)
bands[id_col] = (bands[id_col].astype(str)
                 .str.replace(r"_.*$", "", regex=True)
                 .str.strip()
                 .apply(norm_pid))

indiv_mask_dict = {}
for _, row in bands.iterrows():
    pid = row[id_col]
    mask = np.zeros_like(COMMON_FREQS, dtype=bool)
    for lo, hi in [(row.delta_low, row.delta_high),
    (row.theta_low, row.theta_high),
    (row.alpha_low, row.alpha_high),
    (row.beta_low, row.beta_high),]: 
        mask |= (COMMON_FREQS >= lo) & (COMMON_FREQS <= hi)
    indiv_mask_dict[pid] = mask

# The refions are again defined and stored in a list to be used for regional averaging.
REGIONS = {"F1": ["AF7","AF3","F7","F5","F3","F1"],
           "F2": ["AF4","AF8","F4","F6","F2","F8"],
           "C" : ["Fz","FC1","FCZ","FC2","C1","Cz","C2","CP1","CPZ","CP2","P1","Pz","P2","POZ"],
           "O" : ["PO7","PO8","PO3","PO4","O1","OZ","O2"],
           "P1": ["TP7","CP5","CP3","P7","P5","P3"],
           "P2": ["CP4","CP6","CP8","P4","P6","P8"],
           "T1": ["FT7","FC5","FC3","T7","C5","C3"],
           "T2": ["FC4","FC6","FC8","C4","C6","T8"],}

REGION_NAMES = list(REGIONS)
N_REG = len(REGION_NAMES)

CSV_DIR = os.path.join(OUT_DIR, "alpha_theta_ratio_csv")
os.makedirs(CSV_DIR, exist_ok=True)

# The following loop goes through each participants' extracted epochs file stored in the EPO_DIR path folder.
for fn in sorted(os.listdir(EPO_DIR)):
    if not fn.endswith("_segments-epo.fif"):
        continue

    pid = norm_pid(fn.split("_")[0].replace("sub-", ""))
    if pid not in indiv_mask_dict:
        print(f"Participant {pid} was skipped as they had no indiv mask");  continue
    
    # Reading the epochs of the current participant and taking the metadata. 
    print(f"Currently working on subject {pid}")
    epochs = mne.read_epochs(os.path.join(EPO_DIR, fn),
                             preload=True, verbose="ERROR")
    epochs.metadata = epochs.metadata.rename(columns={"block_idx": "block"})
    meta = epochs.metadata.reset_index(drop=True)

    # The individualised alpha and theta masks are created based on the individualised frequency bands.
    row_bands = bands.loc[bands[id_col] == pid].squeeze()
    alpha_mask = (COMMON_FREQS >= row_bands.alpha_low) & (COMMON_FREQS <= row_bands.alpha_high)
    theta_mask = (COMMON_FREQS >= row_bands.theta_low) & (COMMON_FREQS <= row_bands.theta_high)

    # Freqs and cycles are defined and the practice epochs are identified.
    freqs = COMMON_FREQS
    n_cycles = freqs / 2.0
    pr_idx = np.where(meta.kind == "practice")[0]

    # The regions are defined based on the maps and the available channels in the current participant's data.
    ch_ix      = {ch: i for i, ch in enumerate(epochs.ch_names)}
    reg_to_ix  = [[ch_ix[c] for c in REGIONS[r] if c in ch_ix] for r in REGION_NAMES]

    # A list to store the rows of the CSV file is created.
    csv_rows = []

    # This loop will go over the participants practice epochs to calculate the alpha/theta ratio per region and timepoint and save it to the CSV file.
    for ix in pr_idx:
        blk = int(meta.block.iloc[ix])

        # The TFR is run on the practice epoch to get the power data.
        tfr_p = epochs[ix:ix+1].compute_tfr("morlet", 
                                            freqs=freqs, 
                                            n_cycles=n_cycles,
                                            output="power", 
                                            decim=DECIM, 
                                            n_jobs=N_JOBS)
        pow_p   = tfr_p.data[0]                   
        times   = tfr_p.times - tfr_p.times[0]    

        # The power is averaged for each identified region.
        pow_reg = np.stack([
            pow_p[ix_list].mean(axis=0) if ix_list else np.nan
            for ix_list in reg_to_ix
        ], axis=0)

        # The power in each region is averaged across 14 time bins of 5 seconds each for each frequency.
        for tp in range(14):
            t0, t1 = 5 * tp, 5 * (tp + 1)
            t_sel = np.where((times >= t0) & (times < t1))[0]
            if t_sel.size == 0:
                continue

            # Then in each region and time bin the ATR is calculated by taking the mean power in 
            # the individualised alpha band divided by the mean power in the individualised theta band.
            for r_i, r_name in enumerate(REGION_NAMES):
                alpha_pow = pow_reg[r_i, alpha_mask][:, t_sel].mean()
                theta_pow = pow_reg[r_i, theta_mask][:, t_sel].mean()
                ratio = alpha_pow / theta_pow if theta_pow > 0 else np.nan

                # Lastly, the results and the metadata are appended to the list.
                csv_rows.append({"participant_id" : pid,
                                 "group" : meta.group.iloc[ix],
                                 "block" : blk,
                                 "timepoint" : tp + 1,
                                 "rating" : meta.rating.iloc[ix],
                                 "region": r_name,
                                 "alpha_theta_ratio" : ratio})
        # After writing the data to the list the tfr of the epoch is removed from memory to avoid overload.
        del tfr_p, pow_p, pow_reg
        gc.collect()

    #Lastly, a csv file is created from the complete list of the current participant.
    df_csv = pd.DataFrame(csv_rows)
    csv_path = os.path.join(CSV_DIR, f"sub-{pid}_alpha_theta_ratio.csv")
    df_csv.to_csv(csv_path, index=False)
    print("saved to", csv_path)
    del epochs
    gc.collect()

print("All participants are done")